In [8]:
import kfp

from typing import NamedTuple
import kfp.components as comp
from kfp import compiler, dsl
from kfp import dsl
from kfp.components import InputPath, OutputPath
from kubernetes.client.models import V1EnvVar

In [9]:
def GetRenScores(output_path: OutputPath(str), mlpipeline_metrics_path: OutputPath('Metrics')):
    import requests
    import random
    import json
    import pandas as pd
    from datetime import datetime
    # Get Scores
    
    
    
    url =  "http://influx-api-ren-prototype.apps.paas-dev.psnc.pl/api/measurement/data?measurements=renewability"
    headers = {}
    payload = {}
    response = requests.request("GET", url, headers = headers, data = payload)
    data_json = response.json()

    list_dicts = []
    for dict_ in data_json:
        try:
            value = dict_["fields"]["percentage"]
            time = dict_["tags"]["time_predicition"]
            name_asset = dict_["tags"]["asset_name"]
            ren_dict = {
                "ds": time,
                "value":value,
                "asset_name": name_asset
            }

            list_dicts.append(ren_dict)
        except:
            continue
    data_dicts = pd.DataFrame(list_dicts)

    score_1 = data_dicts[data_dicts.asset_name == "building1"].value.tolist()[-1]
    time_1 = data_dicts[data_dicts.asset_name == "building1"].ds.tolist()[-1]
    score_2 = data_dicts[data_dicts.asset_name == "building2"].value.tolist()[-1]
    time_2 = data_dicts[data_dicts.asset_name == "building1"].ds.tolist()[-1]
    
    print("Last REN Score Value Time")
    print("---------")
    print(time_1)
    print("---------")
    print(time_2)
    
    
    metrics = {
        "metrics":[{
            "name": "score_1",
            "numberValue": score_1
        },
        {
            "name": "score_2",
            "numberValue": score_2
        }]
    }
    data_dict= {
        "building1": score_1,
        "building2": score_2
    }
    with open(mlpipeline_metrics_path, 'w') as f:
        json.dump(metrics, f)
    
    with open(output_path, "w") as out_file:
        json.dump(data_dict, out_file)
    
    

In [10]:
def GetGridInfo(output_assets_path: OutputPath(str), output_demand_path : OutputPath(str)):
    
    import requests
    import json
    
    #GET INFORMATION FROM ASSETS
    
    url_assets = "http://api-ren-prototype.apps.paas-dev.psnc.pl/api/assets/"

    payload={}
    headers = {}

    response = requests.request("GET", url_assets, headers=headers, data=payload)
    
    dict_assets = {"id_asset": [], "name_asset": []}
    for item in response.json(): 
        if item["type"]["name"] == "building":
            dict_assets["id_asset"] = dict_assets["id_asset"] + [item["id"]]
            dict_assets["name_asset"] = dict_assets["name_asset"] + [item["name"]]
    
    
    # GET INFORMATION FROM DEMANDS DEFINITION
    
    url = "http://api-ren-prototype.apps.paas-dev.psnc.pl/api/demandRequests/definition"
    payload={}
    headers = {}

    response = requests.request("GET", url, headers=headers, data=payload)
    
    for demand_response in response.json():
        if demand_response["action_type"] == "DECREASE":
                dict_demands = {
                    "id": demand_response["id"],
                    "action": demand_response["action"],
                    "message": demand_response["message"],
                    "ext": None,
                    "action_type": "DECREASE",
                    "information_tile_id": None
                }
                
    with open(output_assets_path, "w") as out_file:
        json.dump(dict_assets, out_file)
    
    with open(output_demand_path, "w") as out_file:
        json.dump(dict_demands, out_file)

    

In [11]:
def CheckRenScore(asset_name,input_score_path: InputPath(str), min_value = 70) -> bool:
    import json 
    
    with open(input_score_path) as data_file:
        data_str = json.load(data_file)
    #data_str = json.loads(data_str)
    
    value_asset = data_str[asset_name]
    
    print(value_asset)
    
    if float(value_asset) < float(min_value):
        return True
    else:
        return False
    

In [12]:
def CalculateRecommendScore(asset_name,asset_path: InputPath(str), demand_path: InputPath(str)):
    
    import json 
    import requests
    import pandas as pd
    import maya
    
    t_now = maya.now()
    t_update = t_now.add(hours = 1)
    
    with open(asset_path) as data_file:
        data_assets = json.load(data_file)
    
    data_assets = pd.DataFrame(data_assets)
    
    id_asset = data_assets[data_assets.name_asset == asset_name].id_asset.tolist()[0]
    
    with open(demand_path) as data_file:
        data_demand = json.load(data_file)
    
    
    url = "http://api-ren-prototype.apps.paas-dev.psnc.pl/api/demandRequests/"
    
    t_now = maya.now()
    t_end = t_now.add(hours = 1)
    
    dict_to = {
      "asset_id": id_asset,
      "demand_definition": data_demand,
      "demand_start": t_now.epoch,
      "demand_stop": t_end.epoch,
      "demand_update": t_update.epoch
    }
    
    headers = {
            "Content-Type": "application/json"
        }
    
    response = requests.request("POST", url, headers=headers, data=json.dumps(dict_to))
    
    if response.status_code != 200:
        print(response.status_code)
        print(response.text)
        print(dict_to)
        raise ValueError
    
    

In [13]:
def REN_Forecast_Test_Pipeline():
    env_var = V1EnvVar(name='HOME', value='/tmp')
    get_ren_op = comp.create_component_from_func(
        GetRenScores, packages_to_install = ["requests", "pandas"], output_component_file = "ren_scores_op_component.yaml")
    get_grid_info_op = comp.create_component_from_func(
        GetGridInfo, packages_to_install = ["requests"], output_component_file = "get_info_recommend.yaml")
    check_recommend = comp.create_component_from_func(
        CheckRenScore, packages_to_install = [], output_component_file = "check_op_component.yaml")
    recommend_op = comp.create_component_from_func(
        CalculateRecommendScore, packages_to_install = ["requests", "pandas", "maya"], output_component_file = "recommend_component.yaml")
    
    # Get Data from Influx Tasks
    get_ren_task = (get_ren_op().add_env_variable(env_var)
                    .set_memory_request('1Gi')
                    .set_memory_limit('2Gi')
                    .set_cpu_request('2')
                    .set_cpu_limit('4'))
    get_grid_task = (get_grid_info_op().add_env_variable(env_var)
                    .set_memory_request('1Gi')
                    .set_memory_limit('2Gi')
                    .set_cpu_request('2')
                    .set_cpu_limit('4'))
    check_b1_task = (check_recommend("building1", get_ren_task.output, 23.5)
                     .set_memory_request('1Gi')
                    .set_memory_limit('2Gi')
                    .set_cpu_request('2')
                    .set_cpu_limit('4'))
    check_b2_task = (check_recommend("building2", get_ren_task.output, 20)
                     .set_memory_request('1Gi')
                    .set_memory_limit('2Gi')
                    .set_cpu_request('2')
                    .set_cpu_limit('4'))
    with dsl.Condition(check_b1_task.output == True):
        get_b1_recommend = (recommend_op("building1", get_grid_task.outputs["output_assets"], get_grid_task.outputs["output_demand"]).add_env_variable(env_var).after(get_ren_task)
                            .set_memory_request('1Gi')
                                .set_memory_limit('2Gi')
                                .set_cpu_request('2')
                                .set_cpu_limit('4'))
    with dsl.Condition(check_b2_task.output == True):
        get_b2_recommend = (recommend_op("building2",get_grid_task.outputs["output_assets"], get_grid_task.outputs["output_demand"]).add_env_variable(env_var).after(get_ren_task)
                            .set_memory_request('1Gi')
                    .set_memory_limit('2Gi')
                    .set_cpu_request('2')
                    .set_cpu_limit('4'))
    #get_temperature_task = get_measurement_op("current_temperature").add_env_variable(env_var)
    


compiler.Compiler().compile(pipeline_func = REN_Forecast_Test_Pipeline, package_path ="Recommend_Pipeline.yaml")

In [14]:
print("Success")

Success


In [15]:
list_ = ["1665061925", "1665061949", "1665061970", "1665062023", "1665062068", "1665062208", "1665062337", "1665062361", "1665062387", "1665062534", "1665062580", "1665062590", "1665062636", "1665062666", "1665062691", "1665062730", "1665062799", "1665062908", "1665063058", "1665063060", "1665063061", "1665063112", "1665063228", "1665063269", "1665063333", "1665063342", "1665063362", "1665063407", "1665063528", "1665063604", "1665063763", "1665063808", "1665063877", "1665063983", "1665064832", "1665065731", "1665066633", "1665067541", "1665068441", "1665069336", "1665070247", "1665071144", "1665072034", "1665072941", "1665073838", "1665074740", "1665075637", "1665076537", "1665077438", "1665078345", "1665079245", "1665080141", "1665081047", "1665081941", "1665082843", "1665083753", "1665084637", "1665085546", "1665086451", "1665087364", "1665088249", "1665089150", "1665090062", "1665090942", "1665091847", "1665092747", "1665093649", "1665094542", "1665095444", "1665096349", "1665097243", "1665098147", "1665099050", "1665099940", "1665061925", "1665061949", "1665061970", "1665062023", "1665062068", "1665062208", "1665062337", "1665062361", "1665062387", "1665062534", "1665062580", "1665062590", "1665062636", "1665062666", "1665062691", "1665062730", "1665062799", "1665062908", "1665063058", "1665063060", "1665063061", "1665063112", "1665063228", "1665063269", "1665063333", "1665063342", "1665063362", "1665063407", "1665063528", "1665063604", "1665063763", "1665063808", "1665063877", "1665063983", "1665064832", "1665065731", "1665066633", "1665067541", "1665068441", "1665069336", "1665070247", "1665071144", "1665072034", "1665072941", "1665073838", "1665074740", "1665075637", "1665076537", "1665077438", "1665078345", "1665079245", "1665080141", "1665081047", "1665081941", "1665082843", "1665083753", "1665084637", "1665085546", "1665086451", "1665087364", "1665088249", "1665089150", "1665090062", "1665090942", "1665091847", "1665092747", "1665093649", "1665094542", "1665095444", "1665096349", "1665097243", "1665098147", "1665099050", "1665099940", "1665061925", "1665062038", "1665062216", "1665062234", "1665062337", "1665062361", "1665062387", "1665062534", "1665062580", "1665062590", "1665062636", "1665062666", "1665062691", "1665062730", "1665062799", "1665062908", "1665063058", "1665063060", "1665063061", "1665063112", "1665063228", "1665063269", "1665063333", "1665063342", "1665063362", "1665063407", "1665063528", "1665063604", "1665063763", "1665063808", "1665063877", "1665063983", "1665064832", "1665065731", "1665066633", "1665067541", "1665068441", "1665069336", "1665070247", "1665071144", "1665072034", "1665072941", "1665073838", "1665074740", "1665075637", "1665076537", "1665077438", "1665078345", "1665079245", "1665080141", "1665081047", "1665081941", "1665082843", "1665083753", "1665084637", "1665085546", "1665086451", "1665087364", "1665088249", "1665089150", "1665090062", "1665090942", "1665091847", "1665092747", "1665093649", "1665094542", "1665095444", "1665096349", "1665097243", "1665098147", "1665099050", "1665099940", "1665061925", "1665062038", "1665062216", "1665062234", "1665062337", "1665062361", "1665062387", "1665062534", "1665062580", "1665062590", "1665062636", "1665062666", "1665062691", "1665062730", "1665062799", "1665062908", "1665063058", "1665063060", "1665063061", "1665063112", "1665063228", "1665063269", "1665063333", "1665063342", "1665063362", "1665063407", "1665063528", "1665063604", "1665063763", "1665063808", "1665063877", "1665063983", "1665064832", "1665065731", "1665066633", "1665067541", "1665068441", "1665069336", "1665070247", "1665071144", "1665072034", "1665072941", "1665073838", "1665074740", "1665075637", "1665076537", "1665077438", "1665078345", "1665079245", "1665080141", "1665081047", "1665081941", "1665082843", "1665083753", "1665084637", "1665085546", "1665086451", "1665087364", "1665088249", "1665089150", "1665090062", "1665090942", "1665091847", "1665092747", "1665093649", "1665094542", "1665095444", "1665096349", "1665097243", "1665098147", "1665099050", "1665099940", "1665061925", "1665062038", "1665062216", "1665062234", "1665062337", "1665062361", "1665062387", "1665062534", "1665062580", "1665062590", "1665062636", "1665062666", "1665062691", "1665062730", "1665062799", "1665062908", "1665063058", "1665063060", "1665063061", "1665063112", "1665063228", "1665063269", "1665063333", "1665063342", "1665063362", "1665063407", "1665063528", "1665063604", "1665063763", "1665063808", "1665063877", "1665063983", "1665064832", "1665065731", "1665066633", "1665067541", "1665068441", "1665069336", "1665070247", "1665071144", "1665072034", "1665072941", "1665073838", "1665074740", "1665075637", "1665076537", "1665077438", "1665078345", "1665079245", "1665080141", "1665081047", "1665081941", "1665082843", "1665083753", "1665084637", "1665085546", "1665086451", "1665087364", "1665088249", "1665089150", "1665090062", "1665090942", "1665091847", "1665092747", "1665093649", "1665094542", "1665095444", "1665096349", "1665097243", "1665098147", "1665099050", "1665099940", "1665100854", "1665101751", "1665102642", "1665103547", "1665104452", "1665105349", "1665106255", "1665107140", "1665108052", "1665108941", "1665109842", "1665110747", "1665111653", "1665112558", "1665113450", "1665114351", "1665115256", "1665116149", "1665117065", "1665117948", "1665118844", "1665119750", "1665120666", "1665121543", "1665122445", "1665123348", "1665124268", "1665125141", "1665126051", "1665126948", "1665127848", "1665128750", "1665129663", "1665130546", "1665131477", "1665132356", "1665133248", "1665134165", "1665135053", "1665135959", "1665136859", "1665137944", "1665138635", "1665139534", "1665146741", "1665147640", "1665148541", "1665149441", "1665150345", "1665151254", "1665152139", "1665153047", "1665153940", "1665154850", "1665155747", "1665156646", "1665157543", "1665158445", "1665159353", "1665160250", "1665161145", "1665162054", "1665162952", "1665163844", "1665164751", "1665165652", "1665166541", "1665167455", "1665168355", "1665169248", "1665170154", "1665171058", "1665171950", "1665172858", "1665173742", "1665174644", "1665175550", "1665176457", "1665177342", "1665178245", "1665179158", "1665180061", "1665180947", "1665181847", "1665182749", "1665183679", "1665184554", "1665185451", "1665186352", "1665100854", "1665101751", "1665102642", "1665103547", "1665104452", "1665105349", "1665106255", "1665107140", "1665108052", "1665108941", "1665109842", "1665110747", "1665111653", "1665112558", "1665113450", "1665114351", "1665115256", "1665116149", "1665117065", "1665117948", "1665118844", "1665119750", "1665120666", "1665121543", "1665122445", "1665123348", "1665124268", "1665125141", "1665126051", "1665126948", "1665127848", "1665128750", "1665129663", "1665130546", "1665131477", "1665132356", "1665133248", "1665134165", "1665135053", "1665135959", "1665136859", "1665137944", "1665138635", "1665139534", "1665146741", "1665147640", "1665148541", "1665149441", "1665150345", "1665151254", "1665152139", "1665153047", "1665153940", "1665154850", "1665155747", "1665156646", "1665157543", "1665158445", "1665159353", "1665160250", "1665161145", "1665162054", "1665162952", "1665163844", "1665164751", "1665165652", "1665166541", "1665167455", "1665168355", "1665169248", "1665170154", "1665171058", "1665171950", "1665172858", "1665173742", "1665174644", "1665175550", "1665176457", "1665177342", "1665178245", "1665179158", "1665180061", "1665180947", "1665181847", "1665182749", "1665183679", "1665184554", "1665185451", "1665186352", "1665100854", "1665101751", "1665102642", "1665103547", "1665104452", "1665105349", "1665106255", "1665107140", "1665108052", "1665108941", "1665109842", "1665110747", "1665111653", "1665112558", "1665113450", "1665114351", "1665115256", "1665116149", "1665117065", "1665117948", "1665118844", "1665119750", "1665120666", "1665121543", "1665122445", "1665123348", "1665124268", "1665125141", "1665126051", "1665126948", "1665127848", "1665128750", "1665129663", "1665130546", "1665131477", "1665132356", "1665133248", "1665134165", "1665135053", "1665135959", "1665136859", "1665137944", "1665138635", "1665139534", "1665146741", "1665147640", "1665148541", "1665149441", "1665150345", "1665151254", "1665152139", "1665153047", "1665153940", "1665154850", "1665155747", "1665156646", "1665157543", "1665158445", "1665159353", "1665160250", "1665161145", "1665162054", "1665162952", "1665163844", "1665164751", "1665165652", "1665166541", "1665167455", "1665168355", "1665169248", "1665170154", "1665171058", "1665171950", "1665172858", "1665173742", "1665174644", "1665175550", "1665176457", "1665177342", "1665178245", "1665179158", "1665180061", "1665180947", "1665182749", "1665183679", "1665184554", "1665185451", "1665186352", "1665100854", "1665101751", "1665102642", "1665103547", "1665104452", "1665105349", "1665106255", "1665107140", "1665108052", "1665108941", "1665109842", "1665110747", "1665111653", "1665112558", "1665113450", "1665114351", "1665115256", "1665116149", "1665117065", "1665117948", "1665118844", "1665119750", "1665120666", "1665121543", "1665122445", "1665123348", "1665124268", "1665125141", "1665126051", "1665126948", "1665127848", "1665128750", "1665129663", "1665130546", "1665131477", "1665132356", "1665133248", "1665134165", "1665135053", "1665135959", "1665136859", "1665137944", "1665138635", "1665139534", "1665146741", "1665147640", "1665149441", "1665150345", "1665151254", "1665152139", "1665153047", "1665153940", "1665154850", "1665155747", "1665156646", "1665157543", "1665158445", "1665159353", "1665160250", "1665161145", "1665162054", "1665162952", "1665163844", "1665164751", "1665165652", "1665166541", "1665167455", "1665168355", "1665169248", "1665170154", "1665171058", "1665171950", "1665172858", "1665173742", "1665174644", "1665175550", "1665176457", "1665177342", "1665178245", "1665179158", "1665180061", "1665180947", "1665182749", "1665183679", "1665184554", "1665185451", "1665186352", "1665100854", "1665101751", "1665102642", "1665103547", "1665104452", "1665105349", "1665106255", "1665107140", "1665108052", "1665108941", "1665109842", "1665110747", "1665111653", "1665112558", "1665113450", "1665114351", "1665115256", "1665116149", "1665117065", "1665117948", "1665118844", "1665119750", "1665120666", "1665121543", "1665122445", "1665123348", "1665124268", "1665125141", "1665126051", "1665126948", "1665127848", "1665128750", "1665129663", "1665130546", "1665131477", "1665132356", "1665133248", "1665134165", "1665135053", "1665135959", "1665136859", "1665137944", "1665138635", "1665139534", "1665146741", "1665147640", "1665149441", "1665150345", "1665151254", "1665152139", "1665153047", "1665153940", "1665154850", "1665155747", "1665156646", "1665157543", "1665158445", "1665159353", "1665160250", "1665161145", "1665162054", "1665162952", "1665163844", "1665164751", "1665165652", "1665166541", "1665167455", "1665168355", "1665169248", "1665170154", "1665171058", "1665171950", "1665172858", "1665173742", "1665174644", "1665175550", "1665176457", "1665177342", "1665178245", "1665179158", "1665180061", "1665180947", "1665182749", "1665183679", "1665184554", "1665185451", "1665186352", "1665187268", "1665188160", "1665189054", "1665189980", "1665190860", "1665191758", "1665192663", "1665193556", "1665194500", "1665195377", "1665196267", "1665197178", "1665198070", "1665199016", "1665199884", "1665200761", "1665201673", "1665202598", "1665203484", "1665204392", "1665205271", "1665206170", "1665207059", "1665208012", "1665208911", "1665209759", "1665210673", "1665211558", "1665212488", "1665213397", "1665214264", "1665215185", "1665216069", "1665216980", "1665217865", "1665218767", "1665219709", "1665220569", "1665221509", "1665222366", "1665223270", "1665224171", "1665225067", "1665225991", "1665226865", "1665227766", "1665228665", "1665229601", "1665230470", "1665231417", "1665232281", "1665233159", "1665234112", "1665234997", "1665235867", "1665236772", "1665237653", "1665239458", "1665240365", "1665241269", "1665242185", "1665243067", "1665243957", "1665244890", "1665245770", "1665246681", "1665247555", "1665248468", "1665249401", "1665250277", "1665251142", "1665252951", "1665253853", "1665254747", "1665255681", "1665256537", "1665257442", "1665258344", "1665259267", "1665260148", "1665261042", "1665261953", "1665262901", "1665263744", "1665264649", "1665265544", "1665266449", "1665267346", "1665268253", "1665269166", "1665270073", "1665270961", "1665271873", "1665272747", "1665187268", "1665188160", "1665189054", "1665189980", "1665190860", "1665191758", "1665192663", "1665193556", "1665194500", "1665195377", "1665196267", "1665197178", "1665198070", "1665199016", "1665199884", "1665200761", "1665201673", "1665202598", "1665203484", "1665204392", "1665205271", "1665206170", "1665207059", "1665208012", "1665208911", "1665209759", "1665210673", "1665211558", "1665212488", "1665213397", "1665215185", "1665216069", "1665216980", "1665217865", "1665218767", "1665219709", "1665220569", "1665221509", "1665222366", "1665223270", "1665224171", "1665225067", "1665225991", "1665226865", "1665227766", "1665228665", "1665229601", "1665230470", "1665231417", "1665232281", "1665233159", "1665234112", "1665234997", "1665235867", "1665236772", "1665237653", "1665239458", "1665240365", "1665241269", "1665243067", "1665243957", "1665244890", "1665245770", "1665246681", "1665247555", "1665248468", "1665249401", "1665250277", "1665251142", "1665252951", "1665253853", "1665254747", "1665255681", "1665256537", "1665257442", "1665258344", "1665259267", "1665260148", "1665261042", "1665261953", "1665262901", "1665263744", "1665264649", "1665265544", "1665266449", "1665267346", "1665268253", "1665269166", "1665270073", "1665270961", "1665271873", "1665272747", "1665187268", "1665188160", "1665189054", "1665189980", "1665190860", "1665191758", "1665192663", "1665193556", "1665194500", "1665195377", "1665196267", "1665197178", "1665198070", "1665199016", "1665199884", "1665200761", "1665201673", "1665202598", "1665203484", "1665204392", "1665205271", "1665206170", "1665207059", "1665208012", "1665208911", "1665209759", "1665210673", "1665211558", "1665212488", "1665213397", "1665214264", "1665215185", "1665216069", "1665216980", "1665217865", "1665218767", "1665219709", "1665220569", "1665221509", "1665222366", "1665223270", "1665224171", "1665225067", "1665225991", "1665226865", "1665227766", "1665228665", "1665229601", "1665230470", "1665231417", "1665232281", "1665233159", "1665234112", "1665234997", "1665235867", "1665236772", "1665237653", "1665238560", "1665239458", "1665240365", "1665241269", "1665242185", "1665243067", "1665243957", "1665244890", "1665245770", "1665246681", "1665247555", "1665248468", "1665249401", "1665250277", "1665251142", "1665252951", "1665253853", "1665254747", "1665255681", "1665256537", "1665257442", "1665258344", "1665259267", "1665260148", "1665261042", "1665261953", "1665262901", "1665263744", "1665264649", "1665265544", "1665266449", "1665267346", "1665268253", "1665269166", "1665270073", "1665270961", "1665271873", "1665272747", "1665187268", "1665188160", "1665189054", "1665189980", "1665190860", "1665191758", "1665192663", "1665193556", "1665194500", "1665195377", "1665196267", "1665197178", "1665198070", "1665199016", "1665199884", "1665200761", "1665201673", "1665202598", "1665203484", "1665204392", "1665205271", "1665206170", "1665207059", "1665208012", "1665208911", "1665209759", "1665210673", "1665211558", "1665212488", "1665213397", "1665214264", "1665215185", "1665216069", "1665216980", "1665217865", "1665219709", "1665220569", "1665221509", "1665222366", "1665223270", "1665224171", "1665225067", "1665225991", "1665226865", "1665227766", "1665228665", "1665229601", "1665230470", "1665231417", "1665232281", "1665233159", "1665234112", "1665234997", "1665235867", "1665236772", "1665237653", "1665238560", "1665239458", "1665240365", "1665241269", "1665242185", "1665243067", "1665243957", "1665244890", "1665245770", "1665246681", "1665247555", "1665248468", "1665249401", "1665250277", "1665251142", "1665252951", "1665253853", "1665254747", "1665255681", "1665256537", "1665257442", "1665258344", "1665259267", "1665260148", "1665261042", "1665261953", "1665262901", "1665263744", "1665264649", "1665265544", "1665266449", "1665267346", "1665268253", "1665269166", "1665270073", "1665270961", "1665271873", "1665272747", "1665187268", "1665188160", "1665189054", "1665189980", "1665190860", "1665191758", "1665192663", "1665193556", "1665194500", "1665195377", "1665196267", "1665197178", "1665198070", "1665199016", "1665199884", "1665200761", "1665201673", "1665202598", "1665203484", "1665204392", "1665205271", "1665206170", "1665207059", "1665208911", "1665209759", "1665210673", "1665211558", "1665212488", "1665213397", "1665214264", "1665215185", "1665216069", "1665216980", "1665217865", "1665219709", "1665220569", "1665221509", "1665222366", "1665223270", "1665224171", "1665225067", "1665225991", "1665226865", "1665227766", "1665228665", "1665229601", "1665230470", "1665231417", "1665232281", "1665233159", "1665234112", "1665234997", "1665235867", "1665236772", "1665237653", "1665238560", "1665239458", "1665240365", "1665241269", "1665242185", "1665243067", "1665243957", "1665244890", "1665245770", "1665246681", "1665247555", "1665248468", "1665249401", "1665250277", "1665251142", "1665252951", "1665253853", "1665254747", "1665255681", "1665256537", "1665257442", "1665258344", "1665259267", "1665260148", "1665261042", "1665261953", "1665262901", "1665263744", "1665264649", "1665265544", "1665266449", "1665267346", "1665268253", "1665269166", "1665270073", "1665270961", "1665271873", "1665272747", "1665351449", "1665351938", "1665352840", "1665353741", "1665354650", "1665355533", "1665356439", "1665357338", "1665358231", "1665359146", "1665351449", "1665351938", "1665352840", "1665353741", "1665354650", "1665355533", "1665356439", "1665357338", "1665358231", "1665359146", "1665351449", "1665351938", "1665352840", "1665353741", "1665354650", "1665355533", "1665356439", "1665357338", "1665358231", "1665359146", "1665351449", "1665351938", "1665352840", "1665353741", "1665354650", "1665355533", "1665356439", "1665357338", "1665358231", "1665359146", "1665351449", "1665351938", "1665352840", "1665353741", "1665354650", "1665355533", "1665356439", "1665357338", "1665358231", "1665359146"]

In [16]:
len(list_)

1322